## Creating a mesh

We can also use the Splatter wrapper class to take an existing nerfstudio model and create a mesh!
1. **mesh:** creates a mesh via TSDF fusion

2. **query_mesh:** uses the trained model to query the mesh and returns a similarity map

3. **plot_mesh:** enables plotting of mesh features



In [1]:
import os, sys
from pathlib import Path
from collab_splats.wrapper import Splatter, SplatterConfig
import pyvista as pv

# pv.start_xvfb()


Set paths to the file for running splats

In [2]:
base_dir = Path('/workspace/fieldwork-data/')
session_dir = base_dir / "rats/2024-07-11/SplatsSD"

# Make the configuration 
splatter_config = SplatterConfig(
    file_path=session_dir / "C0119.MP4",
    method='rade-features',
    frame_proportion=0.25, # Use 25% of the frames within the video (or default to minimum 300 frames)
)

# Initialize the Splatter class
splatter = Splatter(splatter_config)

# Call these to populate the splatter with paths (probably a better way to do this --> maybe save out config)
splatter.preprocess()
splatter.extract_features()

transforms.json already exists at /workspace/fieldwork-data/rats/2024-07-11/environment/C0119/preproc/transforms.json
To rerun preprocessing, set overwrite=True
Output already exists for rade-features
To rerun feature extraction, set overwrite=True


### Create a mesh

We can create a mesh by calling the ```mesh()``` method. Under the hood, this runs TSDF fusion creating an integrated volume. 

In [13]:
mesher_kwargs = {
    'depth_name': "median_depth",
    'depth_trunc': 1.0, # Should be between 1.0 and 3.0
    'voxel_size': 0.01, 
    'normals_name': "normals",
    'features_name': "distill_features", 
    'sdf_trunc': 0.03,
    'clean_repair': True,
    'align_floor': False,
}

splatter.mesh(
    mesher_type="Open3DTSDFFusion",
    mesher_kwargs=mesher_kwargs,
    overwrite=True
)


Available runs:
[0] 2025-07-11_171420
Initializing mesher Open3DTSDFFusion


TypeError: Open3DTSDFFusion.__init__() got an unexpected keyword argument 'align_floor'

### Plot the mesh!

We can use the splatter function ```plot_mesh``` to visualize given attributes of the mesh. The inherent attributes are RGB and Normals

In [ ]:
splatter.plot_mesh(attribute="RGB")

Number of points: 590469
Number of cells: 1165391
Bounds: BoundsTuple(x_min=-1.172857642173767, x_max=0.9473705291748047, y_min=-0.3601168692111969, y_max=1.4726790189743042, z_min=-1.0063749551773071, z_max=-0.23880182206630707)


Widget(value='<iframe src="http://localhost:42691/index.html?ui=P_0x7246aaa9c280_2&reconnect=auto" class="pyvi…

### Using semantic queries 

The mesh contains semantic features which we can query via positive and negative prompts. The goal of this is to find points that are more similar to the positive prompts compared to the negative prompts

In [10]:
similarity = splatter.query_mesh(
    positive_queries=["tree"],
    negative_queries=["ground", "leaves"],
)

Plot similarity maps

In [11]:
splatter.plot_mesh(attribute=similarity, rgb=False)

Number of points: 590469
Number of cells: 1165391
Bounds: BoundsTuple(x_min=-1.172857642173767, x_max=0.9473705291748047, y_min=-0.3601168692111969, y_max=1.4726790189743042, z_min=-1.0063749551773071, z_max=-0.23880182206630707)


Widget(value='<iframe src="http://localhost:42691/index.html?ui=P_0x7244542b7f40_4&reconnect=auto" class="pyvi…